In [ ]:
!pip install transformers

     |████████████████████████████████| 2.1MB 6.3MB/s 
     |████████████████████████████████| 901kB 37.6MB/s 
     |████████████████████████████████| 3.3MB 52.5MB/s 


In [ ]:
from transformers import TFDistilBertModel
import pandas as pd
from google.colab import drive
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
import json
import numpy as np
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pprint # for pretty printing our device stats

if 'COLAB_TPU_ADDR' not in os.environ:
    print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
    tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    print ('TPU address is', tpu_address)

    with tf.compat.v1.Session(tpu_address) as session:
      devices = session.list_devices()

    print('TPU devices:')
    pprint.pprint(devices)

TPU address is grpc://10.44.201.66:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, -7244579660874423182),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 8589934592, 3151046279729751024),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 8759507057337348006),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, -1509022419331510358),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, -3551633354525984293),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 8690182022618711924),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, -7476826748022715486),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, -7020130666877428780),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))
strategy = tf.distribute.TPUStrategy(resolver)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: grpc://10.44.201.66:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.44.201.66:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
class distilBertModels:
  def __init__(self,path):
    df = pd.read_csv(path)
    self.max_seq_len = 35
    bert_name = 'distilbert-base-uncased'
    # self.tokenizer = AlbertTokenizer.from_pretrained(bert_name,add_special_tokens=True,max_length =self.max_seq_len,pad_to_max_length = True)
    self.train_set,self.test_set = self.prepare_data(df)
    with strategy.scope():
      self.bert = TFDistilBertModel.from_pretrained(bert_name)
      self.bert.trainable = False

  def prepare_data(self,df):
    df['encoded'] = df['encoded'].apply(lambda x: json.loads(x))
    df_tr = df[df['kfold']!=0].reset_index(drop=True)
    df_ts = df[df['kfold']==0].reset_index(drop=True)
    enc_tr = list(df_tr['encoded'])
    enc_ts = list(df_ts['encoded'])
    inp_ids_tr = [np.array(k[0]) for k in enc_tr]
    att_mks_tr = [np.array(k[1]) for k in enc_tr]
    inp_ids_ts = [np.array(k[0]) for k in enc_ts]
    att_mks_ts = [np.array(k[1]) for k in enc_ts]
    x_tr_inp_ids = tf.cast(tf.squeeze(tf.stack(inp_ids_tr)),dtype=tf.int32)
    x_tr_att_mks = tf.cast(tf.squeeze(tf.stack(att_mks_tr)),dtype=tf.int32)
    x_ts_inp_ids = tf.cast(tf.squeeze(tf.stack(inp_ids_ts)),dtype=tf.int32)
    x_ts_att_mks = tf.cast(tf.squeeze(tf.stack(att_mks_ts)),dtype=tf.int32)

    encoder = LabelEncoder()
    y_tr = np_utils.to_categorical(encoder.fit_transform(df_tr.label.values))
    y_ts = np_utils.to_categorical(encoder.fit_transform(df_ts.label.values))

    x_tr = [x_tr_inp_ids,x_tr_att_mks]
    x_ts = [x_ts_inp_ids,x_ts_att_mks]

    return ((x_tr,y_tr),(x_ts,y_ts))

  # def encode_data(self,x):
  #   encoded = self.tokenizer.encode_plus(x,add_special_tokens=True,max_length =self.max_seq_len,padding = 'max_length',truncation=True,return_attention_mask=True,return_tensors='tf')
  #   return encoded
  
  def process(self,ls):
  # for name in ls:
#       his = self.compile_model(name)
#       dfs = []
#       for i in range(len(his)):
#         df = pd.DataFrame.from_dict(his[i].history)
#         df['fold'] = i+1
#         df['epoch'] = [j+1 for j in range(len(df))]
#         dfs.append(df)
#       df_fin = pd.concat(dfs,ignore_index=True)  
#       df_fin.to_csv('/content/drive/MyDrive/Bert/Folds/sent_'+name+'_folds.csv',index=False,header=True)
    for name in ls:
      his = self.compile_model(name)
      df = pd.DataFrame.from_dict(his.history)
      df['epoch'] = [i+1 for i in range(len(df))]
      df.to_csv('/content/drive/MyDrive/DistilBert/sarcasm/sarc_'+name+'.csv',index=False,header=True)
      print("\n------------------ PROCESS FOR MODEL: "+name+' COMPLETED ------------------\n')


  def compile_model(self,name):
    with strategy.scope():
      model = self.create_model(name)
      model.compile(loss='categorical_crossentropy',optimizer = 'adam',metrics=['accuracy'])
    
    
    print('\n------------------ '+name+' ------------------\n')
    model.summary()
    # his = []
    # for i in range(len(self.train_set_folds)):
#       print('\n**************** FOLD '+str(i+1)+' ****************')
    his = model.fit(x = self.train_set[0],y = self.train_set[1],validation_data=(self.test_set[0],self.test_set[1]),epochs=100,callbacks=[EarlyStopping()])
    print('\n**************** TRAINING OF MODEL: '+name+' COMPLETED****************\n')
    return his
    
  def create_model(self,name):
    inputs = tf.keras.layers.Input((self.max_seq_len,),dtype = tf.int32,name="input_ids")
    mask = tf.keras.layers.Input((self.max_seq_len,),dtype = tf.int32,name="attention_mask")

    pred = self.bert([inputs,mask])[0]

    if name == 'DISTILBERT-CONV1D': ## 1
      x = tf.keras.layers.Conv1D(32,3,activation='relu',padding='same')(pred)
      x = tf.keras.layers.AveragePooling1D()(x)
      x = tf.keras.layers.Conv1D(32,3,activation='relu',padding='same')(x)
      x = tf.keras.layers.GlobalAveragePooling1D()(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(20,activation='relu')(x)
      x = tf.keras.layers.Dense(2,activation='softmax')(x)

    
    if name == 'DISTILBERT-CONV1D-BiGRU': ## 2
      x = tf.keras.layers.Conv1D(32,3,activation='relu',padding='same')(pred)
      x = tf.keras.layers.AveragePooling1D()(x)
      x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(16,return_sequences=True,dropout=0.2,recurrent_dropout=0.3))(x)
      x = tf.keras.layers.Flatten()(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(512,activation='relu')(x)
      x = tf.keras.layers.Dense(128,activation='relu')(x)
      x = tf.keras.layers.Dense(32,activation='relu')(x)
      x = tf.keras.layers.Dense(2,activation='softmax')(x)
    
    if name == 'DISTILBERT-CONV1D-BiLSTM': ## 3
      x = tf.keras.layers.Conv1D(32,3,activation='relu',padding='same')(pred)
      x = tf.keras.layers.AveragePooling1D()(x)
      x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16,return_sequences=True,dropout=0.2,recurrent_dropout=0.3))(x)
      x = tf.keras.layers.Flatten()(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(512,activation='relu')(x)
      x = tf.keras.layers.Dense(128,activation='relu')(x)
      x = tf.keras.layers.Dense(32,activation='relu')(x)
      x = tf.keras.layers.Dense(2,activation='softmax')(x)    
      
    if name == 'DISTILBERT-CONV1D-BiRNN': ## 4
      x = tf.keras.layers.Conv1D(32,3,activation='relu',padding='same')(pred)
      x = tf.keras.layers.AveragePooling1D()(x)
      x = tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(16,return_sequences=True,dropout=0.2,recurrent_dropout=0.3))(x)
      x = tf.keras.layers.Flatten()(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(512,activation='relu')(x)
      x = tf.keras.layers.Dense(128,activation='relu')(x)
      x = tf.keras.layers.Dense(32,activation='relu')(x)
      x = tf.keras.layers.Dense(2,activation='softmax')(x)

    if name == 'DISTILBERT-SEP-CONV1D': #5
      x = tf.keras.layers.SeparableConv1D(32,3,activation='relu',padding='same')(pred)
      x = tf.keras.layers.AveragePooling1D()(x)
      x = tf.keras.layers.SeparableConv1D(32,3,activation='relu',padding='same')(x)
      x = tf.keras.layers.AveragePooling1D()(x)
      x = tf.keras.layers.Flatten()(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(256,activation='relu')(x)
      x = tf.keras.layers.Dense(64,activation='relu')(x)
      x = tf.keras.layers.Dense(16,activation='relu')(x)
      x = tf.keras.layers.Dense(2,activation='softmax')(x)
      

    if name == 'DISTILBERT-SEP-CONV1D-BiGRU': ## 6
      x = tf.keras.layers.SeparableConv1D(32,3,activation='relu',padding='same')(pred)
      x = tf.keras.layers.MaxPooling1D()(x)
      x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(16,return_sequences=True,dropout=0.2,recurrent_dropout=0.3))(x)
      x = tf.keras.layers.MaxPooling1D()(x)
      x = tf.keras.layers.Flatten()(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(256,activation='relu')(x)
      x = tf.keras.layers.Dense(64,activation='relu')(x)
      x = tf.keras.layers.Dense(16,activation='relu')(x)
      x = tf.keras.layers.Dense(2,activation='softmax')(x)
      
    if name == 'DISTILBERT-SEP-CONV1D-BiRNN': ## 7
      x = tf.keras.layers.SeparableConv1D(32,3,activation='relu',padding='same')(pred)
      x = tf.keras.layers.MaxPooling1D()(x)
      x = tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(16,return_sequences=True,dropout=0.2,recurrent_dropout=0.3))(x)
      x = tf.keras.layers.MaxPooling1D()(x)
      x = tf.keras.layers.Flatten()(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(256,activation='relu')(x)
      x = tf.keras.layers.Dense(64,activation='relu')(x)
      x = tf.keras.layers.Dense(16,activation='relu')(x)
      x = tf.keras.layers.Dense(2,activation='softmax')(x)
      
    if name == 'DISTILBERT-SEP-CONV1D-BiLSTM': ## 8
      x = tf.keras.layers.SeparableConv1D(32,3,activation='relu',padding='same')(pred)
      x = tf.keras.layers.MaxPooling1D()(x)
      x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16,return_sequences=True,dropout=0.2,recurrent_dropout=0.3))(x)
      x = tf.keras.layers.MaxPooling1D()(x)
      x = tf.keras.layers.Flatten()(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(256,activation='relu')(x)
      x = tf.keras.layers.Dense(64,activation='relu')(x)
      x = tf.keras.layers.Dense(16,activation='relu')(x)
      x = tf.keras.layers.Dense(2,activation='softmax')(x)
      
      
    if name == 'DISTILBERT-BiGRU': ## 9
      x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(16,return_sequences=True,dropout=0.2,recurrent_dropout=0.3))(pred)
      x = tf.keras.layers.MaxPooling1D()(x)
      x = tf.keras.layers.Flatten()(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(256,activation='relu')(x)
      x = tf.keras.layers.Dense(64,activation='relu')(x)
      x = tf.keras.layers.Dense(16,activation='relu')(x)
      x = tf.keras.layers.Dense(2,activation='softmax')(x)
      
    if name == 'DISTILBERT-BiLSTM': ## 10
      x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16,return_sequences=True,dropout=0.2,recurrent_dropout=0.3))(pred)
      x = tf.keras.layers.MaxPooling1D()(x)
      x = tf.keras.layers.Flatten()(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(256,activation='relu')(x)
      x = tf.keras.layers.Dense(64,activation='relu')(x)
      x = tf.keras.layers.Dense(16,activation='relu')(x)
      x = tf.keras.layers.Dense(2,activation='softmax')(x)
    
    if name == 'DISTILBERT-BiRNN': ## 11
      x = tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(16,return_sequences=True,dropout=0.2,recurrent_dropout=0.3))(pred)
      x = tf.keras.layers.MaxPooling1D()(x)
      x = tf.keras.layers.Flatten()(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(256,activation='relu')(x)
      x = tf.keras.layers.Dense(64,activation='relu')(x)
      x = tf.keras.layers.Dense(16,activation='relu')(x)
      x = tf.keras.layers.Dense(2,activation='softmax')(x)
    
      
    return tf.keras.Model([inputs,mask],x)




    

  

In [ ]:
bert = distilBertModels('/content/drive/MyDrive/sarcasm data/tr_sarc_fin_distilbert_trans_inps.csv')

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_projector', 'vocab_transform', 'vocab_layer_norm', 'activation_13']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [ ]:
bert.process(['DISTILBERT-CONV1D','DISTILBERT-CONV1D-BiGRU','DISTILBERT-CONV1D-BiLSTM','DISTILBERT-CONV1D-BiRNN','DISTILBERT-SEP-CONV1D','DISTILBERT-SEP-CONV1D-BiGRU'])
# bert.process(['DISTILBERT-SEP-CONV1D-BiRNN','DISTILBERT-SEP-CONV1D-BiLSTM','DISTILBERT-BiGRU','DISTILBERT-BiLSTM','DISTILBERT-BiRNN'])
# {1:'DISTILBERT-CONV1D',2:'DISTILBERT-CONV1D-BiGRU',3:'DISTILBERT-CONV1D-BiLSTM',4:'DISTILBERT-CONV1D-BiRNN',5:'DISTILBERT-SEP-CONV1D',6:'DISTILBERT-SEP-CONV1D-BiGRU',7:'DISTILBERT-SEP-CONV1D-BiRNN',8:'DISTILBERT-SEP-CONV1D-BiLSTM',9:'DISTILBERT-BiGRU',10:'DISTILBERT-BiLSTM',11:'DISTILBERT-BiRNN'}

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...

7350/7350 [==============================] - ETA: 0s - loss: 0.4011 - accuracy: 0.8162WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


7350/7350 [==============================] - 155s 20ms/step - loss: 0.4011 - accuracy: 0.8162 - val_loss: 0.3425 - val_accuracy: 0.8473
Epoch 2/100
7350/7350 [==============================] - 137s 19ms/step - loss: 0.3453 - accuracy: 0.8472 - val_loss: 0.3357 - val_accuracy: 0.8474
Epoch 3/100
7350/7350 [==============================] - 137s 19ms/step - loss: 0.3322 - accuracy: 0.8536 - val_loss: 0.3230 - val_accuracy: 0.8556
Epoch 4/100
7350/7350 [==============================] - 136s 18ms/step - loss: 0.3214 - accuracy: 0.8585 - val_loss: 0.3154 - val_accuracy: 0.8607
Epoch 5/100
7350/7350 [==============================] - 135s 18ms/step - loss: 0.3155 - accuracy: 0.8619 - val_loss: 0.3116 - val_accuracy: 0.8615
Epoch 6/100
7350/7350 [==============================] - 137s 19ms/step - loss: 0.3091 - accuracy: 0.8651 - val_loss: 0.3068 - val_accuracy: 0.8651
Epoch 7/100
7350/7350 [==============================] - 137s 19ms/step - loss: 0.3047 - accuracy: 0.8670 - val_loss: 0.3086


------------------ DISTILBERT-SEP-CONV1D-BiGRU ------------------

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 35)]         0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 35)]         0                                            
__________________________________________________________________________________________________
tf_distil_bert_model (TFDistilB TFBaseModelOutput(la 66362880    input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
separable_conv1d_2 (Sepa

7350/7350 [==============================] - ETA: 0s - loss: 0.3965 - accuracy: 0.8185WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


7350/7350 [==============================] - 168s 21ms/step - loss: 0.3965 - accuracy: 0.8185 - val_loss: 0.3391 - val_accuracy: 0.8461
Epoch 2/100
7350/7350 [==============================] - 147s 20ms/step - loss: 0.3417 - accuracy: 0.8487 - val_loss: 0.3257 - val_accuracy: 0.8535
Epoch 3/100
7350/7350 [==============================] - 147s 20ms/step - loss: 0.3303 - accuracy: 0.8534 - val_loss: 0.3170 - val_accuracy: 0.8572
Epoch 4/100
7350/7350 [==============================] - 148s 20ms/step - loss: 0.3173 - accuracy: 0.8601 - val_loss: 0.3109 - val_accuracy: 0.8614
Epoch 5/100
7350/7350 [==============================] - 147s 20ms/step - loss: 0.3128 - accuracy: 0.8614 - val_loss: 0.3133 - val_accuracy: 0.8616

**************** TRAINING OF MODEL: DISTILBERT-SEP-CONV1D-BiGRU COMPLETED****************


------------------ PROCESS FOR MODEL: DISTILBERT-SEP-CONV1D-BiGRU COMPLETED ------------------

